# References

https://www.kaggle.com/code/mtszkw/technical-indicators-for-trading-stocks
https://www.kaggle.com/code/thebrownviking20/everything-you-can-do-with-a-time-series
https://www.diva-portal.org/smash/get/diva2:1775077/FULLTEXT01.pdf
https://pub.towardsai.net/predicting-stock-prices-using-arima-fourier-transforms-and-technical-indicators-with-deep-43a164859683


## Import dependencies

In [14]:
! pip install -r requirements.txt


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import pandas as pd
import gdown

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import numpy as np                                    # For matrices!
import matplotlib.pyplot as plt                       # To visualize
from sklearn.linear_model import LinearRegression     # For the regression itself
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.preprocessing import OneHotEncoder       # To convert discrete strings to vectors!
from sklearn.preprocessing import normalize           # For normalizing
import seaborn as sns                                 # For plots

## Change default chart layout

In [3]:
# Show charts when running kernel
init_notebook_mode(connected=True)

# Change default background color for all visualizations
layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(250,250,250,0.8)')
fig = go.Figure(layout=layout)
templated_fig = pio.to_templated(fig)
pio.templates['my_template'] = templated_fig.layout.template
pio.templates.default = 'my_template'

## Download datasets

In [6]:
# SnP500
gdown.download("https://docs.google.com/uc?id=1KveL-W2L6YxO-1NKn0n6FeGTKLWgvAL4", "ES_continuous_adjusted_1min_2011_2018.txt", quiet=False)

# Nasdaq
gdown.download("https://docs.google.com/uc?id=1NyHRoU2YiaCKIhy7afUTuAmC0haWK1Ny", "NQ_continuous_adjusted_1min_2011_2018.txt", quiet=False)

Downloading...
From (uriginal): https://docs.google.com/uc?id=1KveL-W2L6YxO-1NKn0n6FeGTKLWgvAL4
From (redirected): https://docs.google.com/uc?id=1KveL-W2L6YxO-1NKn0n6FeGTKLWgvAL4&confirm=t&uuid=c1c58d40-2190-4d65-9e0b-2d2fb2f90342
To: c:\Users\jdhru\OneDrive\Desktop\Quanthon\quanthon-2023\ES_continuous_adjusted_1min_2011_2018.txt
100%|██████████| 158M/158M [00:25<00:00, 6.14MB/s] 
Downloading...
From (uriginal): https://docs.google.com/uc?id=1NyHRoU2YiaCKIhy7afUTuAmC0haWK1Ny
From (redirected): https://docs.google.com/uc?id=1NyHRoU2YiaCKIhy7afUTuAmC0haWK1Ny&confirm=t&uuid=c9466b7d-5a8f-4d6b-8ac4-4fdaaa393e3a
To: c:\Users\jdhru\OneDrive\Desktop\Quanthon\quanthon-2023\NQ_continuous_adjusted_1min_2011_2018.txt
100%|██████████| 147M/147M [00:21<00:00, 6.87MB/s] 


'NQ_continuous_adjusted_1min_2011_2018.txt'

## Read datasets as CSV

In [7]:
ES_1min_raw = pd.read_csv('ES_continuous_adjusted_1min_2011_2018.txt', encoding='latin-1')
ES_5min_raw = pd.read_csv('ES_continuous_adjusted_5min_2011_2018.txt', encoding='latin-1')
ES_30min_raw = pd.read_csv('ES_continuous_adjusted_30min_2011_2018.txt', encoding='latin-1')
ES_1hour_raw = pd.read_csv('ES_continuous_adjusted_1hour_2011_2018.txt', encoding='latin-1')

NQ_1min_raw = pd.read_csv('NQ_continuous_adjusted_1min_2011_2018.txt', encoding='latin-1')
NQ_5min_raw = pd.read_csv('NQ_continuous_adjusted_5min_2011_2018.txt', encoding='latin-1')
NQ_30min_raw = pd.read_csv('NQ_continuous_adjusted_30min_2011_2018.txt', encoding='latin-1')
NQ_1hour_raw = pd.read_csv('NQ_continuous_adjusted_1hour_2011_2018.txt', encoding='latin-1')